In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from random import uniform, seed
from math import sqrt
from dataclasses import dataclass

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "/Users/maria/anaconda3/bin/python"
  * The NumPy version is: "1.24.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: No module named 'numpy.core._multiarray_umath'


In [1]:
@dataclass
class Agent:
    AgentID: int
    Color: str 
    Happy: int 
    x_loc: float
    y_loc: float
    neighbors: []

NameError: name 'dataclass' is not defined

In [ ]:
def findNeighborDistances(Agent_Array):
    for i in range(0, len(Agent_Array)):
        Agent_Array[i].neighbors.clear()
        for j in range(0, len(Agent_Array)):
            Agent_Array[i].neighbors.append((Agent_Array[j].AgentID, sqrt((Agent_Array[i].x_loc-Agent_Array[j].x_loc)**2+(Agent_Array[i].y_loc-Agent_Array[j].y_loc)**2)))

In [ ]:
def assignHappiness(Agent_Array, bias, k):
    for i in range(0, len(Agent_Array)):
        closestNeighbors = sorted(Agent_Array[i].neighbors, key = lambda x: x[1])[1:(k+1)]
        matchedNeighbors = 0
        AgentColor = Agent_Array[i].Color
        for j in range(0, len(closestNeighbors)):
            neighborID = closestNeighbors[j][0]
            for Agent in Agent_Array:
                if Agent.AgentID == neighborID:
                    if Agent.Color==AgentColor:
                        matchedNeighbors = matchedNeighbors + 1
        if(matchedNeighbors/k >= bias):
            Agent_Array[i].Happy = 1
        else:
            Agent_Array[i].Happy = 0

In [ ]:
def decideWhoMoves(Agent_Array):
    N = len(Agent_Array)
    for i in range(0, N):
        if(Agent_Array[i].Happy == 0):
            Agent_Array[i].x_loc=uniform(0, 1)
            Agent_Array[i].y_loc=uniform(0, 1)

In [ ]:
def plotNeighborhood(Agent_Array):
    df = pd.DataFrame( columns=['x_loc', 'y_loc', 'color'])
    i=0
    for Agent in Agent_Array:
        df.loc[i] = [Agent.x_loc, Agent.y_loc, Agent.Color]
        i = i+1
    plt.scatter(df["x_loc"], df["y_loc"], c = df["color"])
    plt.show()

In [ ]:
def checkHappiness(Agent_Array, plots):
    NumHappy = 0
    for Agent in Agent_Array:
        if Agent.Happy == 1:
            NumHappy = NumHappy+1
    if(plots == True):
        print("Percent of happy agents: ", 100*NumHappy/len(Agent_Array), "% \n")
    return 100*NumHappy/len(Agent_Array)

In [ ]:
def runSim(Agent_Array, steps, bias, k, plots):
    if(plots == True):
        print("Iteration 0: \n")
        plotNeighborhood(Agent_Array)
    percHappyVec = []
    for i in range(0, steps):
        findNeighborDistances(Agent_Array)
        assignHappiness(Agent_Array, bias, k)
        decideWhoMoves(Agent_Array)
        if(plots == True):
            print("Iteration ", i+1, " : \n")
        percHappy = checkHappiness(Agent_Array, plots)
        percHappyVec.append(percHappy)
        if(plots == True):
            plotNeighborhood(Agent_Array)
    return percHappyVec

In [ ]:
def setupAndRunSim(steps, bias, k, orangeNum, greenNum, plots):
    Agent_Array = []
    for i in range(0, orangeNum):
        Agent_Array.append(Agent(AgentID=i, Color="orange", Happy=0, x_loc=uniform(0, 1), y_loc=uniform(0, 1), neighbors=[]))
    for i in range(0, greenNum):
        Agent_Array.append(Agent(AgentID=i+250, Color="green", Happy=0, x_loc=uniform(0, 1), y_loc=uniform(0, 1), neighbors=[]))
    finalPercHappyVec = runSim(Agent_Array, steps, bias, k, plots)
    return finalPercHappyVec

In [ ]:
vec = setupAndRunSim(10, 0.51, 10, 249, 249, True)

In [ ]:
def rejectABC_Posterior(samples, test_vec, steps, k, orangeNum, greenNum):
    params = []
    dists = []
    for i in range(0, samples):
        print("On simulation ", i+1, " of ", samples, "\n")
        gen_param = uniform(0, 1)
        gen_vec = setupAndRunSim(steps, gen_param, k, orangeNum, greenNum, False)
        dist = np.linalg.norm(np.array(test_vec)-np.array(gen_vec))
        params.append(gen_param)
        dists.append(dist)
    return params, dists        

In [ ]:
params, dists = rejectABC_Posterior(1000, vec, 10, 10, 249, 249)

In [ ]:
df1 = pd.DataFrame()
df1['param'] = params
df1['dist'] = dists

In [ ]:
df2 = df1.sort_values(by='dist')

In [ ]:
sns.distplot(df2['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
sns.distplot(df2.iloc[:50]['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
sns.distplot(df2.iloc[:100]['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
sns.distplot(df2.iloc[:250]['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
sns.distplot(df2.iloc[:500]['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
sns.distplot(df2.iloc[:750]['param'], hist = False, kde = True, kde_kws = {'linewidth': 3})
plt.xlim(0,1)

In [ ]:
df_top50 = pd.DataFrame({'param': df2.iloc[:50]['param'], "label": "top 50"})
df_top100 = pd.DataFrame({'param': df2.iloc[:100]['param'], "label": "top 100"})
df_top250 = pd.DataFrame({'param': df2.iloc[:250]['param'], "label": "top 250"})
df_top500 = pd.DataFrame({'param': df2.iloc[:500]['param'], "label": "top 500"})
df_top750 = pd.DataFrame({'param': df2.iloc[:750]['param'], "label": "top 750"})
df_top1000 = pd.DataFrame({'param': df2.iloc[:1000]['param'], "label": "top 1000"})

In [ ]:
df_formatted = pd.concat([df_top50, df_top100, df_top250, df_top500, df_top750, df_top1000], axis=0)

In [ ]:
sns.displot(data=df_formatted, x='param', hue='label', kind='kde', fill=False, palette=sns.color_palette('bright')[:6], height=5, aspect=1.5)